Adapted by Rosa Zwart from Jupyter Notebooks found at https://github.com/PPerdomoQ/rare-disease-explainer

# Import Libraries

In [46]:
import pandas as pd

# Load Data

Load all edges of the knowledge graph.

In [47]:
edges = pd.read_csv('output/new_kg_edges.csv', header=0)
edges.head(10)

,id,subject,object,relation_id,relation_label,relation_iri
0,3d6516a6-a46a-4a31-b3a3-26b99382fbaf,FlyBase:FBgn0011676,ENSEMBL:ENSCAFG00000009820,RO:HOM0000017,in orthology relationship with,http://purl.obolibrary.org/obo/RO_HOM0000017
1,5b3867c6-befd-4bad-b61c-97de75ae22c6,HGNC:12012,HGNC:11946,RO:0002434,interacts with,http://purl.obolibrary.org/obo/RO_0002434
2,a276b95d-2137-49a2-84fd-585298f81a1f,HGNC:15516,HP:0002650,RO:0003303,causes condition,http://purl.obolibrary.org/obo/RO_0003303
3,ff4c3b46-0c34-4e73-9754-5351221f4b40,WormBase:WBGene00000065,ENSEMBL:ENSOANG00000007850,RO:HOM0000017,in orthology relationship with,http://purl.obolibrary.org/obo/RO_HOM0000017
4,4f82bdff-cbcb-45b2-95d7-086b142257c9,HGNC:6485,HGNC:329,RO:0002434,interacts with,http://purl.obolibrary.org/obo/RO_0002434
5,48dee90f-a334-459b-8cbc-999bc745c49f,ENSEMBL:ENSECAG00000023844,ENSEMBL:ENSGALG00000042388,RO:HOM0000017,in orthology relationship with,http://purl.obolibrary.org/obo/RO_HOM0000017
6,657f2866-d1a5-405b-bd02-dec3964150d5,MGI:3621491,MP:0011966,RO:0003303,causes condition,http://purl.obolibrary.org/obo/RO_0003303
7,25a138d1-4498-4864-96c0-12c1f91ae493,MGI:1859649,ENSEMBL:ENSACAG00000001940,RO:HOM0000017,in orthology relationship with,http://purl.obolibrary.org/obo/RO_HOM0000017
8,439999a4-578f-4641-b022-90d2ea670eed,ClinVarVariant:803793,MONDO:0010679,RO:0003303,causes condition,http://purl.obolibrary.org/obo/RO_0003303
9,8b79b610-3a7b-4f87-af8c-47010bc60e69,PomBase:SPAC27F1.02c,RGD:1559479,RO:HOM0000017,in orthology relationship with,http://purl.obolibrary.org/obo/RO_HOM0000017


Extract the relevant columns and rename these columns.

In [48]:
edges = edges[['subject', 'relation_label', 'object']]
edges.rename(columns={'subject': 'head', 'relation_label': 'relation', 'object': 'tail'}, inplace=True)
edges.head(10)

,head,relation,tail
0,FlyBase:FBgn0011676,in orthology relationship with,ENSEMBL:ENSCAFG00000009820
1,HGNC:12012,interacts with,HGNC:11946
2,HGNC:15516,causes condition,HP:0002650
3,WormBase:WBGene00000065,in orthology relationship with,ENSEMBL:ENSOANG00000007850
4,HGNC:6485,interacts with,HGNC:329
5,ENSEMBL:ENSECAG00000023844,in orthology relationship with,ENSEMBL:ENSGALG00000042388
6,MGI:3621491,causes condition,MP:0011966
7,MGI:1859649,in orthology relationship with,ENSEMBL:ENSACAG00000001940
8,ClinVarVariant:803793,causes condition,MONDO:0010679
9,PomBase:SPAC27F1.02c,in orthology relationship with,RGD:1559479


Load all nodes of the knowledge graph.

In [49]:
nodes = pd.read_csv('output/new_kg_nodes.csv', header=0)
nodes['index_id'] = nodes.index
nodes.head(10)

,id,label,iri,semantic,index_id
0,MP:0020358,abnormal inhibitory synapse morphology,http://purl.obolibrary.org/obo/MP_0020358,phenotype,0
1,HP:0000670,Carious teeth,http://purl.obolibrary.org/obo/HP_0000670,phenotype,1
2,ENSEMBL:ENSCAFG00000017522,ENSEMBL:ENSCAFG00000017522,http://ensembl.org/id/ENSCAFG00000017522,gene,2
3,ZFIN:ZDB-GENE-070705-188,chrm1b,http://zfin.org/ZDB-GENE-070705-188,gene,3
4,HP:0000544,External ophthalmoplegia,http://purl.obolibrary.org/obo/HP_0000544,phenotype,4
5,ZFIN:ZDB-GENE-040426-1146,traf3ip1,http://zfin.org/ZDB-GENE-040426-1146,gene,5
6,MP:0001900,impaired synaptic plasticity,http://purl.obolibrary.org/obo/MP_0001900,phenotype,6
7,ClinVarVariant:650539,NM_004006.2(DMD):c.6439-10_8217+32103dup,http://www.ncbi.nlm.nih.gov/clinvar/variation/...,variant,7
8,ZFIN:ZDB-GENE-070228-4,drp2,http://zfin.org/ZDB-GENE-070228-4,gene,8
9,HGNC:7611,MYOD1,https://www.genenames.org/data/gene-symbol-rep...,gene,9


Extract the relevant columns and rename these columns.

In [50]:
nodes = nodes[['index_id', 'id', 'semantic', 'label']]
nodes.head(10)

,index_id,id,semantic,label
0,0,MP:0020358,phenotype,abnormal inhibitory synapse morphology
1,1,HP:0000670,phenotype,Carious teeth
2,2,ENSEMBL:ENSCAFG00000017522,gene,ENSEMBL:ENSCAFG00000017522
3,3,ZFIN:ZDB-GENE-070705-188,gene,chrm1b
4,4,HP:0000544,phenotype,External ophthalmoplegia
5,5,ZFIN:ZDB-GENE-040426-1146,gene,traf3ip1
6,6,MP:0001900,phenotype,impaired synaptic plasticity
7,7,ClinVarVariant:650539,variant,NM_004006.2(DMD):c.6439-10_8217+32103dup
8,8,ZFIN:ZDB-GENE-070228-4,gene,drp2
9,9,HGNC:7611,gene,MYOD1


Convert categorical to numerical values for the semantic groups.

In [51]:
nodes['semantic_id'] = nodes.semantic.astype('category').cat.codes
nodes.head(10)

,index_id,id,semantic,label,semantic_id
0,0,MP:0020358,phenotype,abnormal inhibitory synapse morphology,8
1,1,HP:0000670,phenotype,Carious teeth,8
2,2,ENSEMBL:ENSCAFG00000017522,gene,ENSEMBL:ENSCAFG00000017522,5
3,3,ZFIN:ZDB-GENE-070705-188,gene,chrm1b,5
4,4,HP:0000544,phenotype,External ophthalmoplegia,8
5,5,ZFIN:ZDB-GENE-040426-1146,gene,traf3ip1,5
6,6,MP:0001900,phenotype,impaired synaptic plasticity,8
7,7,ClinVarVariant:650539,variant,NM_004006.2(DMD):c.6439-10_8217+32103dup,10
8,8,ZFIN:ZDB-GENE-070228-4,gene,drp2,5
9,9,HGNC:7611,gene,MYOD1,5


Merge nodes and edges in order to join semantic classes of nodes with heads and tails of the edges.

In [52]:
kg_df = pd.merge(edges, nodes,
                 left_on='head', right_on='id',
                 how='left')
kg_df = kg_df[['head', 'label', 'semantic_id', 'index_id', 'relation', 'tail']]
kg_df.rename(columns={'label': 'label_head', 'semantic_id': 'class_head', 'index_id': 'index_head'}, inplace=True)
kg_df.head(10)

,head,label_head,class_head,index_head,relation,tail
0,FlyBase:FBgn0011676,Nos,5,9594,in orthology relationship with,ENSEMBL:ENSCAFG00000009820
1,HGNC:12012,TPM3,5,1813,interacts with,HGNC:11946
2,HGNC:15516,XYLT1,5,3228,causes condition,HP:0002650
3,WormBase:WBGene00000065,act-3,5,3272,in orthology relationship with,ENSEMBL:ENSOANG00000007850
4,HGNC:6485,LAMA5,5,4641,interacts with,HGNC:329
5,ENSEMBL:ENSECAG00000023844,ENSEMBL:ENSECAG00000023844,5,9822,in orthology relationship with,ENSEMBL:ENSGALG00000042388
6,MGI:3621491,Dmd<mdx>/Dmd<mdx> [C57BL/10ScSn-Dmd<mdx>/J],6,4598,causes condition,MP:0011966
7,MGI:1859649,Ctnnal1,5,5773,in orthology relationship with,ENSEMBL:ENSACAG00000001940
8,ClinVarVariant:803793,NM_004006.3(DMD):c.9950_9951del (p.Lys3317fs),10,1815,causes condition,MONDO:0010679
9,PomBase:SPAC27F1.02c,cdc8,5,3477,in orthology relationship with,RGD:1559479


In [53]:
kg_df = pd.merge(kg_df, nodes,
                 left_on='tail', right_on='id',
                 how='left')
kg_df = kg_df[['head', 'label_head', 'class_head', 'index_head', 'relation', 'tail', 'label', 'semantic_id', 'index_id']]
kg_df.rename(columns={'label': 'label_tail', 'semantic_id': 'class_tail', 'index_id': 'index_tail'}, inplace=True)
kg_df.head(10)

,head,label_head,class_head,index_head,relation,tail,label_tail,class_tail,index_tail
0,FlyBase:FBgn0011676,Nos,5,9594,in orthology relationship with,ENSEMBL:ENSCAFG00000009820,ENSEMBL:ENSCAFG00000009820,5,7362
1,HGNC:12012,TPM3,5,1813,interacts with,HGNC:11946,TNNI2,5,7360
2,HGNC:15516,XYLT1,5,3228,causes condition,HP:0002650,Scoliosis,8,2102
3,WormBase:WBGene00000065,act-3,5,3272,in orthology relationship with,ENSEMBL:ENSOANG00000007850,ENSEMBL:ENSOANG00000007850,5,2284
4,HGNC:6485,LAMA5,5,4641,interacts with,HGNC:329,AGRN,5,2205
5,ENSEMBL:ENSECAG00000023844,ENSEMBL:ENSECAG00000023844,5,9822,in orthology relationship with,ENSEMBL:ENSGALG00000042388,A0A3Q2TS00,5,9573
6,MGI:3621491,Dmd<mdx>/Dmd<mdx> [C57BL/10ScSn-Dmd<mdx>/J],6,4598,causes condition,MP:0011966,abnormal auditory brainstem response waveform ...,8,839
7,MGI:1859649,Ctnnal1,5,5773,in orthology relationship with,ENSEMBL:ENSACAG00000001940,ENSEMBL:ENSACAG00000001940,5,2803
8,ClinVarVariant:803793,NM_004006.3(DMD):c.9950_9951del (p.Lys3317fs),10,1815,causes condition,MONDO:0010679,Duchenne muscular dystrophy,3,4672
9,PomBase:SPAC27F1.02c,cdc8,5,3477,in orthology relationship with,RGD:1559479,Tpm2,5,6456


In [54]:
nodes.drop('index_id', axis=1, inplace=True)
nodes.head(10)

,id,semantic,label,semantic_id
0,MP:0020358,phenotype,abnormal inhibitory synapse morphology,8
1,HP:0000670,phenotype,Carious teeth,8
2,ENSEMBL:ENSCAFG00000017522,gene,ENSEMBL:ENSCAFG00000017522,5
3,ZFIN:ZDB-GENE-070705-188,gene,chrm1b,5
4,HP:0000544,phenotype,External ophthalmoplegia,8
5,ZFIN:ZDB-GENE-040426-1146,gene,traf3ip1,5
6,MP:0001900,phenotype,impaired synaptic plasticity,8
7,ClinVarVariant:650539,variant,NM_004006.2(DMD):c.6439-10_8217+32103dup,10
8,ZFIN:ZDB-GENE-070228-4,gene,drp2,5
9,HGNC:7611,gene,MYOD1,5


In [55]:
kg_df.head(15)

,head,label_head,class_head,index_head,relation,tail,label_tail,class_tail,index_tail
0,FlyBase:FBgn0011676,Nos,5,9594,in orthology relationship with,ENSEMBL:ENSCAFG00000009820,ENSEMBL:ENSCAFG00000009820,5,7362
1,HGNC:12012,TPM3,5,1813,interacts with,HGNC:11946,TNNI2,5,7360
2,HGNC:15516,XYLT1,5,3228,causes condition,HP:0002650,Scoliosis,8,2102
3,WormBase:WBGene00000065,act-3,5,3272,in orthology relationship with,ENSEMBL:ENSOANG00000007850,ENSEMBL:ENSOANG00000007850,5,2284
4,HGNC:6485,LAMA5,5,4641,interacts with,HGNC:329,AGRN,5,2205
5,ENSEMBL:ENSECAG00000023844,ENSEMBL:ENSECAG00000023844,5,9822,in orthology relationship with,ENSEMBL:ENSGALG00000042388,A0A3Q2TS00,5,9573
6,MGI:3621491,Dmd<mdx>/Dmd<mdx> [C57BL/10ScSn-Dmd<mdx>/J],6,4598,causes condition,MP:0011966,abnormal auditory brainstem response waveform ...,8,839
7,MGI:1859649,Ctnnal1,5,5773,in orthology relationship with,ENSEMBL:ENSACAG00000001940,ENSEMBL:ENSACAG00000001940,5,2803
8,ClinVarVariant:803793,NM_004006.3(DMD):c.9950_9951del (p.Lys3317fs),10,1815,causes condition,MONDO:0010679,Duchenne muscular dystrophy,3,4672
9,PomBase:SPAC27F1.02c,cdc8,5,3477,in orthology relationship with,RGD:1559479,Tpm2,5,6456


# Preparation E2V Data

Encode string values of relation labels as numeric values.

In [56]:
kg_df['type'] = pd.factorize(kg_df['relation'])[0]
kg_df.head(10)

,head,label_head,class_head,index_head,relation,tail,label_tail,class_tail,index_tail,type
0,FlyBase:FBgn0011676,Nos,5,9594,in orthology relationship with,ENSEMBL:ENSCAFG00000009820,ENSEMBL:ENSCAFG00000009820,5,7362,0
1,HGNC:12012,TPM3,5,1813,interacts with,HGNC:11946,TNNI2,5,7360,1
2,HGNC:15516,XYLT1,5,3228,causes condition,HP:0002650,Scoliosis,8,2102,2
3,WormBase:WBGene00000065,act-3,5,3272,in orthology relationship with,ENSEMBL:ENSOANG00000007850,ENSEMBL:ENSOANG00000007850,5,2284,0
4,HGNC:6485,LAMA5,5,4641,interacts with,HGNC:329,AGRN,5,2205,1
5,ENSEMBL:ENSECAG00000023844,ENSEMBL:ENSECAG00000023844,5,9822,in orthology relationship with,ENSEMBL:ENSGALG00000042388,A0A3Q2TS00,5,9573,0
6,MGI:3621491,Dmd<mdx>/Dmd<mdx> [C57BL/10ScSn-Dmd<mdx>/J],6,4598,causes condition,MP:0011966,abnormal auditory brainstem response waveform ...,8,839,2
7,MGI:1859649,Ctnnal1,5,5773,in orthology relationship with,ENSEMBL:ENSACAG00000001940,ENSEMBL:ENSACAG00000001940,5,2803,0
8,ClinVarVariant:803793,NM_004006.3(DMD):c.9950_9951del (p.Lys3317fs),10,1815,causes condition,MONDO:0010679,Duchenne muscular dystrophy,3,4672,2
9,PomBase:SPAC27F1.02c,cdc8,5,3477,in orthology relationship with,RGD:1559479,Tpm2,5,6456,0


Extract the relevant columns for applying the Edge2Vec method.

In [57]:
edge2vec_df = kg_df.drop(['head', 'relation', 'tail'], axis=1)
edge2vec_df['index_head'] = edge2vec_df['index_head'].apply(str)
edge2vec_df['index_tail'] = edge2vec_df['index_tail'].apply(str)
edge2vec_df.head(10)

,label_head,class_head,index_head,label_tail,class_tail,index_tail,type
0,Nos,5,9594,ENSEMBL:ENSCAFG00000009820,5,7362,0
1,TPM3,5,1813,TNNI2,5,7360,1
2,XYLT1,5,3228,Scoliosis,8,2102,2
3,act-3,5,3272,ENSEMBL:ENSOANG00000007850,5,2284,0
4,LAMA5,5,4641,AGRN,5,2205,1
5,ENSEMBL:ENSECAG00000023844,5,9822,A0A3Q2TS00,5,9573,0
6,Dmd<mdx>/Dmd<mdx> [C57BL/10ScSn-Dmd<mdx>/J],6,4598,abnormal auditory brainstem response waveform ...,8,839,2
7,Ctnnal1,5,5773,ENSEMBL:ENSACAG00000001940,5,2803,0
8,NM_004006.3(DMD):c.9950_9951del (p.Lys3317fs),10,1815,Duchenne muscular dystrophy,3,4672,2
9,cdc8,5,3477,Tpm2,5,6456,0
